# Speeding up your Neural Network  with Theano and the GPU

## Brief intro on Theano

#### GPU

A decent explanation on the purpose of using GPU in deep learning: [link](https://www.quora.com/Why-are-GPUs-well-suited-to-deep-learning).


#### Theano

[Theano](http://deeplearning.net/software/theano/) is a Python library that lets you define, optimize, and evaluate mathematical expressions, espectially with ones with multi-dimensional arrays (`np.ndarrays`). At it's heart, Theano is a compiler for mathematical expressions in Python, which takes your structures and turn their into very efficient code.

It is not a programming language (you write in Python) but you still need to:
- declare variables and give their types
- build expressions for how to put those variables together
- compile expression graphs to functions in order to use them for computation

## Modify the original implementation

#### Basic Setup

In [ ]:
import numpy as np
import sklearn
import sklearn.datasets
import theano
import theano.tensor as T
import time

train_X, train_y = sklearn.datasets.make_moons(5000, noise=0.40) # Generate datasets

As a remark, the default floating-point data type is `float64`, but in order to use your GPU you must use `float32`.

In [ ]:
theano.config.floatX = 'float32'

Now begin the setup. Check out [Neural Network from scratch](https://bitbucket.org/rootofallevii/neural-network-from-scratch/src/master/) if any of these seems unfamiliar to you.

In [ ]:
ndim = 1000                   # hidden layer contains 1000 nodes
n_examples = len(train_X)     # training set size
nn_input_dim = 2              # dimension of input layer
nn_output_dim = 2             # dimension of output layer

epsilon = np.float32(0.01)    # arbitrary learn rate
reg_lambda = np.float32(0.01) # arbitrary regularization strength

#### Shared Variables

Shared variables behave more like ordinary Python variables. They have an explicit value that is persistent. In contrast, symbolic variables (the "normal" variable in Theano) are not given explicit value until one is assigned on the excecution of a compiled Theano function. 

Symbolic variables can be thought of as representing state for the duration of a single execution. Shared variables on the other hand represent state thta remains in memory for the lifetime of the Python reference.

Obviously, since we want to constantly train and update weight and bias, parameters $W_1, W_2, b_1, b_2$ need to be shared variables; the data $X$ and $y$ also need to be persistent.

In [ ]:
X = theano.shared(train_X.astype('float32'))            # convert to float32
y = theano.shared(np.eye(2)[train_y].astype('float32')) # convert to float32

# We also give these variables names
W1 = theano.shared(np.random.randn(nn_input_dim, ndim).astype('float32'), name='W1')
b1 = theano.shared(np.zeros(ndim).astype('float32'), name='b1')
W2 = theano.shared(np.random.randn(ndim, nn_output_dim).astype('float32'), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim).astype('float32'), name='b2')

#### Forward Propagation

Note that Theano contains a built-in softmax function we can use! `T.nnet` has a range of NN-related functions, including sigmoid, ReLu, and cross-entropy.

In [ ]:
z1 = X.dot(W1) + b1
a1 = T.tanh(z1)
z2 = a1.dot(W2) + b2
y_hat = T.nnet.softmax(z2)

#### Regularization

Operations in Theano are typically vector/matrix-friendly.

In [ ]:
loss_reg = 1./n_examples * reg_lambda/2 * (T.sum(T.sqr(W1)) + T.sum(T.sqr(W2)))

#### Optimization

Built-in cross-entropy loss function!

In [ ]:
loss = T.nnet.categorical_crossentropy(y_hat, y).mean() + loss_reg

#### Prediction

Based on the input, the model outputs the predicted class.

In [ ]:
prediction = T.argmax(y_hat, axis=1)

#### Gradients

The function `T.grad(f, x)` computes the derivative of `f` with respect to variable `x`.

In [ ]:
dW2 = T.grad(loss, W2)
db2 = T.grad(loss, b2)
dW1 = T.grad(loss, W1)
db1 = T.grad(loss, b1)

#### Compile to functions

There is no argument since we always use the same shared variables. You might note there is a delay from Jupyter kernel. This is because internally Theano compiles these into C code.

In [ ]:
forward_prop = theano.function([], y_hat)
calculate_loss = theano.function([], loss)
predict = theano.function([], prediction)

#### Gradient

The option `update=(var, expr)` tells the function to update the variable `var` with the value from `expr`.

In [ ]:
gradient_step = theano.function([], updates=((W2, W2-epsilon*dW2),
                                            (W1, W1-epsilon*dW1),
                                            (b2, b2-epsilon*db2),
                                            (b1, b1-epsilon*db1)))

## Build the model and test it out

#### Wrapper function

The function `var.set_value(val)` function assigns value `val` to shared variable `var`.

In [ ]:
def build_model(num_passes=20000, print_loss=False):
    
    """ Learns parameters for the NN and returns the model.
        - nums_passes: number of iterations.
        - print_loss: True if printing loss for every 1000 iterations. """
    
    np.random.seed(0)
    W1.set_value((np.random.randn(nn_input_dim, ndim) / np.sqrt(nn_input_dim)).astype('float32'))
    b1.set_value(np.zeros(ndim).astype('float32'))
    W2.set_value((np.random.randn(ndim, nn_output_dim) / np.sqrt(ndim)).astype('float32'))
    b2.set_value(np.zeros(nn_output_dim).astype('float32'))
    
    for i in range(0, num_passes):
        gradient_step() # batch gradient descent
        if print_loss and i % 1 == 0:
            print("Loss after iteration %i: %f" %(i, calculate_loss()))

#### Train and visualize

Time to build the NN and plot the decision boundary. Check out [Neural Network from scratch](https://bitbucket.org/rootofallevii/neural-network-from-scratch/src/master/) for explanation on plotting helper function.

In [ ]:
# NN from scratch 
def plot_decision_boundary(pred_func):
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    h = 0.01 
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

In [ ]:
build_model(print_loss=True)
plot_decision_boundary(lambda x: predict(x))
plt.title("Decision boundary for hidden layer size 3")

It is recommended to run this optimized code on an GPU-optimized Amazon EC2 instance.